# Тестовое задание

### Описание данных

В приложении к заданию есть файл database.db. Он представляет собой файл БД на фреймворке sqlite3. Синтаксис SQL, используемый в данной БД почти не отличается от ORACLE DB (более подробно можно почитать тут: https://unetway.com/tutorial/sqlite-syntax).
Запросы к базе можно сохранять сразу в pandas.Dataframe, используя следующий код:

In [ ]:
import pandas as pd
import sqlite3 as sql

# Загрузка файла из GitHub
!wget https://raw.githubusercontent.com/ADMukhametzianov/VTB/main/database.db

In [27]:
query = 'SELECT * FROM pl where client_id = "yr3sFJNQex"'
con = sql.connect('database.db')
df = pd.read_sql_query(sql = query, con=con)
con.close()

df

,client_id,segment,period,product,pl
0,yr3sFJNQex,Массовый,p_1,РКО,6.551622
1,yr3sFJNQex,Массовый,p_1,Депозит,36.038988
2,yr3sFJNQex,Массовый,p_2,РКО,17.283574


   В базе данных хранится 2 таблицы: pl и campaign. Ключами для связи этих таблиц является поле client_id. В таблице pl находится информация о ЧОД (чистый операционный доход) от клиента с продуктом в разрезе периодов. В таблице campaign хранятся данные о прошедшей  между отчетными периодами "p_1" и "p_2" маркетинговой кампании.<br>
Атрибутный состав таблицы pl:
   - client_id - id клиента
   - segment - сегмент клиента внутри СМБ(Средний и малый бизнес)
   - period - период расчета ЧОД банка от продукта, используемого клиентом
   - product - продукт
   - pl - ЧОД в тысячах рублей
   
Пример: строка 0 на срезе выше - в периоде "p_1" за продукт "Кредит" с клиента "yxwtZv530y" массового сегмента банк заработал 0.374924 тыс. рублей.
   
Если в каком-либо периоде информация о клиенте/продукте у клиента отсутствует, значит в этот период клиента в банке не было/клиент продуктом не пользовался.

Атрибутный состав таблицы campaign:
- client_id - id клиента
- product - продукт кампании
- response - статус отклика на предложение
- manager_id - id менеджера, на которого назначен клиент

### Задание

1) Посчитать средний ЧОД по продуктам и количество клиентов с продуктом для каждого сегмента в разрезе периодов.<br>
2) Посчитать средний ЧОД и количество клиентов в каждом сегменте в разрезе периодов.<br>
3) Оценить отток клиентов с продуктов в периоде p_2.<br>
4) Посчитать сумму ЧОД, привлеченную каждым менеджером в ходе маркетинговой кампании<br>
5) Оценить эффективность маркетинговой кампании.

### Задание 1. Посчитать средний ЧОД по продуктам и количество клиентов с продуктом для каждого сегмента в разрезе периодов.<br>

In [8]:
con = sql.connect('database.db')
query = '''
SELECT pl.segment, pl.period, pl.product, round(AVG(pl.pl), 1) AS avg_pl, COUNT(DISTINCT pl.client_id) AS client_count
FROM pl
LEFT JOIN campaign ON pl.client_id = campaign.client_id
GROUP BY pl.segment, pl.period, pl.product
ORDER BY pl.segment, pl.period, pl.product;
'''

# Выполнение запроса и сохранение результатов в DataFrame
result = pd.read_sql_query(query, con)

# Вывод результата
print(result)

# Закрытие соединения с базой данных
con.close()

        segment period           product  avg_pl  client_count
0       Верхний    p_1  Банковская карта    60.6            49
1       Верхний    p_1           Депозит   112.5           254
2       Верхний    p_1            Кредит   236.9           360
3       Верхний    p_1               РКО    22.8          1149
4       Верхний    p_1         Эквайринг    80.5           183
5       Верхний    p_2  Банковская карта    51.8           167
6       Верхний    p_2           Депозит    99.3           283
7       Верхний    p_2            Кредит   248.2           548
8       Верхний    p_2               РКО    22.1          1162
9       Верхний    p_2         Эквайринг    73.0           284
10     Массовый    p_1  Банковская карта    13.3           895
11     Массовый    p_1           Депозит    28.7          3537
12     Массовый    p_1            Кредит    59.2          5247
13     Массовый    p_1               РКО     5.8         16728
14     Массовый    p_1         Эквайринг    19.6       

In [6]:
con = sql.connect('database.db')

# Запрос для получения данных из таблицы pl
query_pl = "SELECT * FROM pl"
df_pl = pd.read_sql_query(query_pl, con)

# Запрос для получения данных из таблицы campaign
# query_campaign = "SELECT * FROM campaign"
# df_campaign = pd.read_sql_query(query_campaign, con)

# Расчет среднего ЧОД по продуктам и количество клиентов с продуктом для каждого сегмента в разрезе периодов
result = df_pl.groupby(['segment', 'period', 'product']).agg({'pl': 'mean', 'client_id': 'nunique'}).reset_index()
result.pl = round(result.pl, 1)
# Вывод результата
print(result)
con.close()

        segment period           product     pl  client_id
0       Верхний    p_1  Банковская карта   60.6         49
1       Верхний    p_1           Депозит  112.5        254
2       Верхний    p_1            Кредит  236.9        360
3       Верхний    p_1               РКО   22.8       1149
4       Верхний    p_1         Эквайринг   80.5        183
5       Верхний    p_2  Банковская карта   51.8        167
6       Верхний    p_2           Депозит   99.3        283
7       Верхний    p_2            Кредит  248.2        548
8       Верхний    p_2               РКО   22.1       1162
9       Верхний    p_2         Эквайринг   73.0        284
10     Массовый    p_1  Банковская карта   13.3        895
11     Массовый    p_1           Депозит   28.7       3537
12     Массовый    p_1            Кредит   59.2       5247
13     Массовый    p_1               РКО    5.8      16728
14     Массовый    p_1         Эквайринг   19.6       2625
15     Массовый    p_2  Банковская карта   11.7       26

### Задание 2. Посчитать средний ЧОД и количество клиентов в каждом сегменте в разрезе периодов.<br>

In [28]:
con = sql.connect('database.db')

query = """
SELECT
    pl.segment,
    pl.period,
    round(AVG(pl.pl), 1) AS avg_pl,
    COUNT(DISTINCT pl.client_id) AS client_count
FROM pl
LEFT JOIN campaign ON pl.client_id = campaign.client_id
GROUP BY pl.segment, pl.period
ORDER BY pl.segment, pl.period;
"""

# Выполнение запроса и сохранение результатов в DataFrame
result = pd.read_sql_query(query, con)

# Вывод результата
print(result)

# Закрытие соединения с базой данных
con.close()

       segment period  avg_pl  client_count
0      Верхний    p_1    79.1          1195
1      Верхний    p_2    89.7          1206
2     Массовый    p_1    19.7         17207
3     Массовый    p_2    23.3         17330
4  Стандартный    p_1    40.3          6061
5  Стандартный    p_2    46.5          6110


### Задание 3. Оценить отток клиентов с продуктов в периоде p_2.<br>

In [41]:
con = sql.connect('database.db')

# SQL запрос для получения информации о количестве уникальных клиентов по продуктам в периодах "p_1" и "p_2"
query = """
SELECT
    pl.product,
    COUNT(DISTINCT CASE WHEN pl.period = 'p_1' THEN pl.client_id END) AS p1_clients,
    COUNT(DISTINCT CASE WHEN pl.period = 'p_2' THEN pl.client_id END) AS p2_clients,
    COUNT(DISTINCT CASE WHEN pl.period = 'p_2' THEN pl.client_id END) - COUNT(DISTINCT CASE WHEN pl.period = 'p_1' THEN pl.client_id END) AS client_diff
FROM pl
GROUP BY pl.product
"""

# Выполнение запроса и сохранение результатов в DataFrame
result = pd.read_sql_query(query, con)
result["client_diff_percent"] = round(result.client_diff/result.p1_clients*100, 2)
# Вывод результата
print(result)

# Закрытие соединения с базой данных
con.close()

            product  p1_clients  p2_clients  client_diff  client_diff_percent
0  Банковская карта        1282        3720         2438               190.17
1           Депозит        5083        6198         1115                21.94
2            Кредит        7478       11264         3786                50.63
3               РКО       23784       23732          -52                -0.22
4         Эквайринг        3770        6298         2528                67.06
